In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importação de bibliotecas


Nessa seção faremos as importações das bibliotecas nescessarias para o projeto:



*   *NumPy:*  é uma biblioteca para a linguagem de programação Python, que suporta o processamento de grandes, multi-dimensionais arranjos e matrizes, juntamente com uma grande coleção de funções matemáticas de alto nível para operar sobre estas matrizes.

*   *TensorFlow:* é uma biblioteca de código aberto para aprendizado de máquina aplicável a uma ampla variedade de tarefas. É um sistema para criação e treinamento de redes neurais para detectar e decifrar padrões e correlações, análogo à forma como humanos aprendem e raciocinam.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.utils import load_img
from keras.metrics import Accuracy, Precision, Recall

#### Definindo hiperparâmetro

Nessa seção fizemos a pré definição dos parâmetros a serem utilizados durante a construção de rede:



*   *Batch size:* Representa o valor da quantidade de imagens que serão analisadas por pacote.
*   *Epochs:* Determina a quantidade de épocas na qual o modelo será treinado antes de finalizar o processo.
*   *Input shape:* Esse input determinará o tamanho que cada imagem terá sendo **Largura x Altura x Quantidade de Cores**







In [3]:
batch_size = 32
epochs = 15
input_shape = (150, 150, 3)  # Tamanho das imagens de entrada

###geradores de dados para pré-processamento

Esse momento será realizada a captação, normalização e divisão entre dados de teste e validação, com a utilização da ferramenta **ImageDataGenerator**, por meio da função **flow_from_directory**.



*   Path
*   target_size
*   Shuffle
*   batch_size
*   Class_mode
*   Subset



In [4]:
data_generator = ImageDataGenerator(rescale=1./255, validation_split=(0.3))
path = '/content/drive/MyDrive/Pasta sem nome/train'

train_generator = data_generator.flow_from_directory(
    path,
    target_size=input_shape[:2],
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = data_generator.flow_from_directory(
    path,
    target_size=input_shape[:2],
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 3664 images belonging to 2 classes.
Found 1568 images belonging to 2 classes.


# Criando parâmetros de CallBack

A criação de **callbacks** por meio de funções presentes no modulo **tensorflow** é um metodo que auxilia na melhor execução de **RN's**



*   ***ModelChechpoint:*** capturam o valor exato de todos os parâmetros (tf. Objetos variáveis) usados ​​por um modelo.
*   ***ReduceLROnPlateau:*** redução de Learning Reating, é uma forma de conferencia de platô local durante os testes, levando a uma melhor metrica de avaliação.
*   ***EarlyStopping:*** realiza a parada do teste caso as metricas não tenham melhora em determinada quantidade de *Epochs*.



In [5]:
filepeath = 'transferlearning_weigths.hdf5'
checkpoint = ModelCheckpoint(filepeath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [6]:
lr_reduce = ReduceLROnPlateau(monitor= 'val_loss', factor=0.8, min_delta= 0.001, patience= 3, verbose=1)

In [7]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience= 5, verbose=1, mode='min')

In [8]:
callbacks = [ lr_reduce, checkpoint, early_stop]

###Construindo o modelo da CNC (REDES NEURAIS CONVOLUCIONAIS)

Dentre os varios metodos de construção de modelos de CNC, foi feito uso de um modelo ***Sequential*** com 7 camadas e redução de parte dos parâmetros usando a função ***Dropout***

*   **Conv2d**: Camada de convolução 2D, esta camada cria um kernel de convolução que é enrolado com entrada de camadas que ajuda a produzir saídas.
*   **MaxPolling2D**: Reduz a amostra da entrada ao longo de suas dimensões espaciais (altura e largura) obtendo o valor máximo em uma janela de entrada (de tamanho definido por pool_size) para cada canal da entrada.
*   **Dropuot**: define aleatoriamente as unidades de entrada como 0 com uma frequência de taxa em cada etapa durante o tempo de treinamento, o que ajuda a evitar o overfitting. As entradas não definidas como 0 são escaladas em 1/(1 - taxa) de modo que a soma de todas as entradas permaneça inalterada.

In [9]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape ))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 36, 36, 128)       0

### COMPILAÇÃO DO MODELO

A compilação do modelo é a parte do processo responsavel pela escolha de Optimizer, perca e as metricas a serem avaliadas durante os testes.

*   **Optimizer**: ***adam*** é um método de descida de gradiente estocástico baseado na estimativa adaptativa de momentos de primeira e segunda ordem.
*   **Loss**: ***binary_crossentropy*** é uma métrica de modelo que rastreia a rotulagem incorreta da classe de dados por um modelo, penalizando o modelo se ocorrerem desvios na probabilidade de classificar os rótulo.
*   **metrics**: ***accuracy,precision,recall*** são metricas de avaliação padronizadas e nescessarias para avaliar qualquer teste.

In [10]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

### TREINO DO MODELO

O treino do modelo, representa a parte do projeto responsavel pelos testes e aprendizado do modelo com base nas metricas pré compiladas

*   **steps_per_epoch**: Número total de etapas (lotes de amostras) antes de declarar uma época concluída e iniciar a próxima época
*   **validation_data**: recebe os comparadores para validação dos testes durante as passagens de época no teste
*   **validation_steps**: Número total de etapas (lotes de amostras) a serem coletadas antes de parar ao executar a validação no final de cada época.
*   **callbacks**: Lista de retornos de chamada a serem aplicados durante a avaliação.

In [11]:
model.fit(train_generator,
          steps_per_epoch=train_generator.samples//batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples//batch_size,
          callbacks = callbacks)

Epoch 1/15
114/114 [==============================] - ETA: 0s - loss: 0.4847 - accuracy: 0.8549 - precision: 0.8525 - recall: 0.8499 
Epoch 1: val_loss improved from -inf to 0.20844, saving model to transferlearning_weigths.hdf5
114/114 [==============================] - 1756s 15s/step - loss: 0.4847 - accuracy: 0.8549 - precision: 0.8525 - recall: 0.8499 - val_loss: 0.2084 - val_accuracy: 0.9228 - val_precision: 0.9205 - val_recall: 0.9228 - lr: 0.0010
Epoch 2/15
114/114 [==============================] - ETA: 0s - loss: 0.1220 - accuracy: 0.9568 - precision: 0.9522 - recall: 0.9548
Epoch 2: val_loss did not improve from 0.20844
114/114 [==============================] - 565s 5s/step - loss: 0.1220 - accuracy: 0.9568 - precision: 0.9522 - recall: 0.9548 - val_loss: 0.1812 - val_accuracy: 0.9388 - val_precision: 0.9399 - val_recall: 0.9375 - lr: 0.0010
Epoch 3/15
114/114 [==============================] - ETA: 0s - loss: 0.1089 - accuracy: 0.9598 - precision: 0.9600 - recall: 0.9587
Ep

###AVALIÇÃO DO MODELO

A avaliação do modelo representa a etapa responsavel por quantificar as metricas avaliativas do modelo testado.

*   **Accuracy**
*   **Precision**
*   **recall**
*   **F1_score**

In [12]:
# Avalie o modelo nos dados de teste
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(validation_generator, verbose=1)

49/49 [==============================] - 63s 1s/step - loss: 0.1552 - accuracy: 0.9630 - precision: 0.9631 - recall: 0.9643


In [13]:
print('Perda de teste:', test_loss)
print('Acurácia de teste:', test_accuracy)
print('Precisão de teste:', test_precision)
print('Teste de Recall', test_recall)

Perda de teste: 0.15521292388439178
Acurácia de teste: 0.9630101919174194
Precisão de teste: 0.9630573391914368
Teste de Recall 0.9642857313156128


In [14]:
f1_score_divisor= test_precision * test_recall
f1_score_dividendo = test_precision + test_recall

In [15]:
f1 = (2*f1_score_divisor) / f1_score_dividendo
f1

0.9636711437956588